<br><br>

## **Import necessary Python libraries and modules**

In [1]:

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch



To use the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html), we will install it with `pip`.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.8 MB/s 
     |████████████████████████████████| 895 kB 26.6 MB/s 
     |████████████████████████████████| 6.5 MB 21.0 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 596 kB 35.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast
from transformers import Trainer, TrainingArguments

<br><br>

## **Set parameters and file paths**

In [4]:
# This is the name of the BERT model that we want to use. 
# We're using DistilBERT to save space (it's a distilled version of the full BERT model), 
# and we're going to use the cased (vs uncased) version.
model_name = 'xlm-roberta-base'  

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'       

# This is the maximum number of tokens in any document sent to BERT.
max_length = 128                                                       

# This is the name of the directory where we'll save our model. You can name it whatever you want.
#cached_model_directory_name = 'ABSA_FineTuning_BERT'  
cached_model_directory_name = 'pol_base'  

In [5]:
#stiamo utlizzando la GPU?
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [6]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 62.5 MB/s 
     |████████████████████████████████| 448 kB 49.7 MB/s 


In [7]:
#pretokenizzazionen in morfemi delle koreane
from konlpy.tag import Okt
def pretok_ko(subset, indice):
  new_ko=[] 
  #count=0
  for index,row in subset.iterrows():
    if index < indice:
      new_ko.append(row['Sentence'])
    else: #quando arrivi all'indice di ko

      okt = Okt()
      li=okt.morphs(str(row['Sentence']))
      stringa=''
      for el in li:
        stringa+=str(el)+' '
      new_ko.append(stringa[:-1])
  subset['Sentence']=new_ko
  return subset
    


In [8]:
test=pd.read_csv("test_def_pol.csv")
test=pretok_ko(test,699)
test

,Unnamed: 0,Sentence,Polarity_Classification
0,0,This is one of the books that my dad read out ...,Positiva
1,1,Garth Nix's 'Mister Monday' was a highly cleve...,Positiva
2,2,"Includes affirmations, visualizations, and pra...",Positiva
3,3,Monday was fantasy,Neutra
4,4,Weird and dangerous creatures are hunting him ...,Neutra
...,...,...,...
1387,1387,"달리는 동안 그 는 이 것 을 작은 열쇠 라고 하는 것 을 갖게 되었고 , 그것 은...",Neutra
1388,1388,아벨라 부터 들어간 기아 순정 이긴 한데 프라이드 는 잘 모르 겠 네요 ㅋㅋ 끝물 ...,Neutra
1389,1389,이전 이나 변함 없다 .,Negativa
1390,1390,[ 3 점 ] 뭐 하자는거지 ?,Negativa


In [9]:
#creiamo le liste di train e test. 1.8k train 250 test  
#train_texts=dev['Sentence']
#train_labels=dev['Emotions']
test_texts=test['Sentence']
test_labels=test['Polarity_Classification']


<br><br>

## **Split the data into training and test sets**

In [10]:
len(test_texts), len(test_labels)

(1392, 1392)

Here's an example of a training label and review:

In [11]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [12]:
def prediction_test(dataset):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels
    tok = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base') # The model_name needs to match our pre-trained model.
    test_texts=dataset['Sentence']
    test_labels=dataset['Polarity_Classification']

    unique_labels={'Neutra', 'Positiva', 'Negativa'}
    label2id={'Negativa': 2, 'Neutra': 0, 'Positiva': 1}
    id2label={2:'Negativa', 0:'Neutra', 1:'Positiva'}  

  
#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=128)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    # The model_name needs to match the name used for the tokenizer above.
    modello = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=len(id2label)).to('cuda')
    training_args=TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
)
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
    
    predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

#len(predicted_labels)

    return print(classification_report(test_labels,predicted_labels))

In [13]:
prediction_test(test)

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    Negativa       0.00      0.00      0.00       460
      Neutra       0.00      0.00      0.00       467
    Positiva       0.33      1.00      0.50       465

    accuracy                           0.33      1392
   macro avg       0.11      0.33      0.17      1392
weighted avg       0.11      0.33      0.17      1392

